In [2]:
# imports
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import statsmodels.formula.api as smf

In [3]:
# read hist ads
hist_df = pd.read_csv('prop_data/ads_historicals.csv')
hist_df['Date'] = pd.to_datetime(hist_df['Date'])
hist_df = hist_df.sort_values(by='Date', ascending=True)

# Function to get the first day of the week
def week_start_date(date):
    return date - pd.to_timedelta(date.dayofweek, unit='d')

# Function to get the first day of the month
def month_start_date(date):
    return date.replace(day=1)

# Apply the functions to the dataframe
hist_df['Week_Start'] = hist_df['Date'].apply(week_start_date)
hist_df['Month_Start'] = hist_df['Date'].apply(month_start_date)

# Now group by the new 'Week_Start' and 'Month_Start' columns to get the ad counts
weekly_ad_count = hist_df.groupby('Week_Start').size().reset_index(name='Total Ads')
monthly_ad_count = hist_df.groupby('Month_Start').size().reset_index(name='Total Ads')

# print median ads per week
print('Median ads per week:', weekly_ad_count['Total Ads'].median())

Median ads per week: 2.5


**Historical Ad Prices**

Q1 Ad Rev: 13,050

Q1 Ads: 29

Per Ad Rate: 450K

In [4]:
# initialize the revenue df

concerts_df = pd.read_excel('data/postcards_revenue_projection.xlsx')
# get the date range
start_date = hist_df['Date'].max()
end_date = concerts_df['Date'].max()
date_range = pd.date_range(start_date, end_date, freq='W')
# create the revenue df
revenue_df = pd.DataFrame(columns=['Week', 'Number of Ads', 'Ad Rate', 'Ad Revenue'])
revenue_df['Week'] = date_range
revenue_df['Number of Ads'] = 0
revenue_df['Ad Rate'] = 0
revenue_df['Ad Revenue'] = 0

revenue_df.head()

,Week,Number of Ads,Ad Rate,Ad Revenue
0,2024-01-28,0,0,0
1,2024-02-04,0,0,0
2,2024-02-11,0,0,0
3,2024-02-18,0,0,0
4,2024-02-25,0,0,0


In [5]:
DAILY_RATE=450000
MEDIAN_ADS=weekly_ad_count['Total Ads'].median()

# calculate the ad rate
revenue_df['Number of Ads'] = MEDIAN_ADS
revenue_df['Ad Rate'] = DAILY_RATE
revenue_df['Ad Revenue'] = revenue_df['Number of Ads'] * revenue_df['Ad Rate']

revenue_df.head()

,Week,Number of Ads,Ad Rate,Ad Revenue
0,2024-01-28,2.5,450000,1125000.0
1,2024-02-04,2.5,450000,1125000.0
2,2024-02-11,2.5,450000,1125000.0
3,2024-02-18,2.5,450000,1125000.0
4,2024-02-25,2.5,450000,1125000.0


**Superbowl week:**

- Feb 7 - Feb 14
- 14 ad spots
- Range 1M - 2M / ad
- Assume avg 1.5M
- Already fully sold out

In [6]:
# manually add superbowl ads (1.5M each 2x/day for feb7 - feb14 2024)
revenue_df.loc[revenue_df['Week'] == '2024-02-04', 'Ad Revenue'] = (1500000 * 7)
revenue_df.loc[revenue_df['Week'] == '2024-02-04', 'Ad Rate'] = 1500000
revenue_df.loc[revenue_df['Week'] == '2024-02-04', 'Number of Ads'] = 7
revenue_df.loc[revenue_df['Week'] == '2024-02-11', 'Ad Revenue'] = (1500000 * 7)
revenue_df.loc[revenue_df['Week'] == '2024-02-11', 'Ad Rate'] = 1500000
revenue_df.loc[revenue_df['Week'] == '2024-02-11', 'Number of Ads'] = 7

**Rig Known Q3 Ads into top of DF**

In [7]:
q2_end = pd.to_datetime('2023-12-31')
# add the following weeks into the revenue df with 0 values for everything
# 2024-01-07
# 2024-01-14
# 2024-01-21

revenue_df = revenue_df.append({'Week': q2_end + pd.DateOffset(weeks=1), 'Number of Ads': 4, 'Ad Rate': 450000, 'Ad Revenue': 1800000}, ignore_index=True)
revenue_df = revenue_df.append({'Week': q2_end + pd.DateOffset(weeks=2), 'Number of Ads': 1, 'Ad Rate': 450000, 'Ad Revenue': 450000}, ignore_index=True)
revenue_df = revenue_df.append({'Week': q2_end + pd.DateOffset(weeks=3), 'Number of Ads': 4, 'Ad Rate': 450000, 'Ad Revenue': 1800000}, ignore_index=True)

# sort by date
revenue_df = revenue_df.sort_values(by='Week', ascending=True)
revenue_df.head()

/var/folders/rs/k8yvpht95zxbr8d7v79msjk00000gn/T/ipykernel_37369/1349296160.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  revenue_df = revenue_df.append({'Week': q2_end + pd.DateOffset(weeks=1), 'Number of Ads': 4, 'Ad Rate': 450000, 'Ad Revenue': 1800000}, ignore_index=True)
/var/folders/rs/k8yvpht95zxbr8d7v79msjk00000gn/T/ipykernel_37369/1349296160.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  revenue_df = revenue_df.append({'Week': q2_end + pd.DateOffset(weeks=2), 'Number of Ads': 1, 'Ad Rate': 450000, 'Ad Revenue': 450000}, ignore_index=True)
/var/folders/rs/k8yvpht95zxbr8d7v79msjk00000gn/T/ipykernel_37369/1349296160.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  revenue_df = revenue_df.append({'Week': 

,Week,Number of Ads,Ad Rate,Ad Revenue
53,2024-01-07,4.0,450000,1800000.0
54,2024-01-14,1.0,450000,450000.0
55,2024-01-21,4.0,450000,1800000.0
0,2024-01-28,2.5,450000,1125000.0
1,2024-02-04,7.0,1500000,10500000.0


In [8]:
# reset index
revenue_df = revenue_df.reset_index(drop=True)
revenue_df.head()

,Week,Number of Ads,Ad Rate,Ad Revenue
0,2024-01-07,4.0,450000,1800000.0
1,2024-01-14,1.0,450000,450000.0
2,2024-01-21,4.0,450000,1800000.0
3,2024-01-28,2.5,450000,1125000.0
4,2024-02-04,7.0,1500000,10500000.0


In [9]:
# Plot the historical ads / week and the projected ads / week as a bar chart
fig = go.Figure()
fig.add_trace(go.Bar(x=weekly_ad_count['Week_Start'], y=weekly_ad_count['Total Ads'], name='Historical Ads'))
fig.add_trace(go.Bar(x=revenue_df['Week'], y=revenue_df['Number of Ads'], name='Projected Ads'))
fig.update_layout(title='Historical and Projected Ads per Week', xaxis_title='Week', yaxis_title='Number of Ads')
fig.show()

In [10]:
# Combine historical and projected and export as excel
combined_df = pd.concat([weekly_ad_count, revenue_df], axis=0)
combined_df = combined_df.sort_values(by='Week', ascending=True)
combined_df = combined_df.reset_index(drop=True)
combined_df.to_excel('dviz/combined_ads.xlsx', index=False)

In [11]:
# export to excel
revenue_df.to_excel('data/ads_revenue_projection.xlsx', index=False)